In [8]:
# !pip install -q openai pandas

import os, json, time, pathlib
import pandas as pd
from openai import OpenAI

In [ ]:
###################################################
OPENAI_API_KEY = "REMOVED FOR SECURITY REASONS"
###################################################

MODEL_NAME = "gpt-4o-mini"  
SLICE = (0, 150)
DRY_RUN = False
PACING_SLEEP = 10.0        

DATA_DIR = pathlib.Path("data")
RESULTS_DIR = pathlib.Path("results/gpt")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

QID_LIST = DATA_DIR / f"run_qids_{SLICE[0]}_{SLICE[1]}.txt" # created by Gemini notebook
QUESTIONS_ONLY_JSONL = DATA_DIR / f"questions_only_{SLICE[0]}_{SLICE[1]}.jsonl"

RESULTS_JSONL = RESULTS_DIR / "results_gpt_medquestions150.jsonl"
RESULTS_CSV   = RESULTS_DIR / "results_gpt_medquestions150.csv"

In [10]:
def append_jsonl(path, record):
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

def load_done_ids(path):
    done = set()
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    qid = json.loads(line).get("qid")
                    if qid: done.add(qid)
                except:
                    pass
    return done

In [11]:
client = OpenAI(api_key=OPENAI_API_KEY)

def ask_gpt(question: str) -> str:
    msgs = [
        {"role": "system", "content": "You are a concise medical Q&A assistant. Answer briefly and directly."},
        {"role": "user", "content": question},
    ]
    resp = client.chat.completions.create(model=MODEL_NAME, messages=msgs, temperature=0)
    return resp.choices[0].message.content.strip()

In [12]:
subset_q = [json.loads(l) for l in open(QUESTIONS_ONLY_JSONL, "r", encoding="utf-8")]

done = load_done_ids(RESULTS_JSONL)
subset_q = [r for r in subset_q if r["qid"] not in done]

print(f"Slice: {SLICE}, dry_run={DRY_RUN}")
if DRY_RUN: subset_q = subset_q[:3]
print("Will ask now:", len(subset_q))

count = errors = 0
t0 = time.time()

for rec in subset_q:
    qid = rec["qid"]
    question = rec["question"]

    pred = ""
    status, err = "ok", None
    t1 = time.time()

    for wait_s in [0, 1, 2]:
        if wait_s: time.sleep(wait_s)
        try:
            pred = ask_gpt(question)
            status, err = "ok", None
            break
        except Exception as e:
            status, err = "error", str(e)
    else:
        errors += 1
        pred = ""

    latency_ms = int((time.time() - t1) * 1000)

    append_jsonl(RESULTS_JSONL, {
        "qid": qid,
        "model": MODEL_NAME,
        "question": question,
        "model_answer": pred,
        "latency_ms": latency_ms,
        "status": status,
        "error": err,
    })

    count += 1
    if count % 10 == 0:
        print(f"Progress: {count}/{len(subset_q)} (~{(time.time()-t0)/60:.1f} min)")

    time.sleep(PACING_SLEEP)

print(f"Completed new entries: {count} | Errors: {errors} | Elapsed min: {(time.time()-t0)/60:.1f}")

Slice: (0, 150), dry_run=False
Will ask now: 150
Progress: 10/150 (~2.0 min)
Progress: 20/150 (~4.1 min)
Progress: 30/150 (~6.1 min)
Progress: 40/150 (~8.1 min)
Progress: 50/150 (~10.1 min)
Progress: 60/150 (~12.1 min)
Progress: 70/150 (~14.0 min)
Progress: 80/150 (~16.0 min)
Progress: 90/150 (~18.1 min)
Progress: 100/150 (~20.1 min)
Progress: 110/150 (~22.1 min)
Progress: 120/150 (~24.1 min)
Progress: 130/150 (~48.6 min)
Progress: 140/150 (~50.6 min)
Progress: 150/150 (~52.7 min)
Completed new entries: 150 | Errors: 0 | Elapsed min: 52.9


In [13]:
df = pd.read_json(RESULTS_JSONL, lines=True)
df.to_csv(RESULTS_CSV, index=False)

print("Saved:")
print(" ", RESULTS_JSONL)
print(" ", RESULTS_CSV)
print("OK rows:", (df["status"] == "ok").sum(), " | ERROR rows:", (df["status"] == "error").sum())
df.head(3)

Saved:
  results\gpt\results_gpt_medquestions150.jsonl
  results\gpt\results_gpt_medquestions150.csv
OK rows: 150  | ERROR rows: 0


,qid,model,question,model_answer,latency_ms,status,error
0,medquestions-500-0000,gpt-4o-mini,What are the genetic changes related to famili...,Familial adenomatous polyposis (FAP) is primar...,2899,ok,NaN
1,medquestions-500-0001,gpt-4o-mini,What are the treatments for Noonan syndrome ?,Treatment for Noonan syndrome is symptomatic a...,3872,ok,NaN
2,medquestions-500-0002,gpt-4o-mini,How to diagnose National Hormone and Pituitary...,To diagnose conditions related to the National...,2565,ok,NaN
